In [1]:
import torch

In [7]:
# 变换1：使用view reshape API
# view reshape使用上是一样的，且不改变原始数据
# 逻辑上可以使用上述API对数据做任意形状的变换，只要满足相乘后的维度一致就可以
# 实际使用中变换其实是要符合物理意义的
a = torch.rand(4, 1, 28, 28)
print(a.shape)

b = a.view(4, 28 * 28)  # b = a.view(4 * 28, 28)  b = a.view(4 * 1, 28, 28) ...
# b = a.reshape(4, 28 * 28)
print(b.shape)
print(a.shape)

torch.Size([4, 1, 28, 28])
torch.Size([4, 784])
torch.Size([4, 1, 28, 28])


In [10]:
# 变换2：使用unsqueeze squeeze API
# 对数据进行扩维或降维操作，可以根据自己的需要引入一些新的概念来管理数据
# 数据本身存储不变，只是数据的理解方式变了

a = torch.rand(4, 1, 28, 28)
print(a.shape)

b = a.unsqueeze(0)  # 在原来的第一维之前扩维，这个的物理意义可以理解为对一个batch_size的图片引入组的概念
print(b.shape)

c = a.unsqueeze(-1)  # 在原来的最后一维之后扩维
print(c.shape)

d = c = a.unsqueeze(4)  # 在原来的第4维之前扩维，含义等同于a.unsqueeze(-1)
print(d.shape)

## 总结
"""
  以上述例子为基础展开
      a.unsqueeze(dim)  dim取值范围 [-a.dim() - 1, a.dim() + 1) 超出这个范围会报错
      a.dim = 0 1 2 3
      扩维规则：
          - 非负数在dim之前扩维
          - 负数在dim之后扩维
"""

torch.Size([4, 1, 28, 28])
torch.Size([1, 4, 1, 28, 28])
torch.Size([4, 1, 28, 28, 1])
torch.Size([4, 1, 28, 28, 1])


In [15]:
# 变换2：unsqueeze API的例子
a = torch.tensor([1.2, 2.3])
print(a)
print(a.shape)

print(a.unsqueeze(-1))
print(a.unsqueeze(-1).shape)

print(a.unsqueeze(0))
print(a.unsqueeze(0).shape)

tensor([1.2000, 2.3000])
torch.Size([2])
tensor([[1.2000],
        [2.3000]])
torch.Size([2, 1])
tensor([[1.2000, 2.3000]])
torch.Size([1, 2])


In [20]:
# 变换2：squeeze API
a = torch.rand(1, 32, 2, 1)

b = a.squeeze()  # 不给参数，把能挤压的全部挤压，即维度为1的全部被挤压
print(b.shape)

c = a.squeeze(0)  # 把第一维挤压掉
print(c.shape)

d = a.squeeze(1)  # 维度不是1的不能被挤压掉
print(d.shape)

torch.Size([32, 2])
torch.Size([32, 2, 1])
torch.Size([1, 32, 2, 1])


In [6]:
# 变换3：expand repeat API 不会改变原来的张量
# 对维度进行扩展，与变换2不同，它不改变数据的维度，而是改变维度上的值
# expand: broadcasting，不会主动复制数据，只在有需要的时候复制，推荐使用
# repeat: memory copied 实实在在复制了数据，参数表示的是复制的次数
a = torch.rand(4, 32, 14, 14)
b = torch.rand(1, 32, 1, 1)

c = b.expand(4, 32, 14, 14)  # 实现的是某一维度从1 -> M，但是不能实现某一维度从 M -> N
print(b.shape)
print(c.shape)

d = b.expand(-1, 32, -1, -1)  # 使用 -1 来代替其他维度的值，因为有时候这个值是计算出来的
print(d.shape)

torch.Size([1, 32, 1, 1])
torch.Size([4, 32, 14, 14])
torch.Size([1, 32, 1, 1])


In [5]:
a = torch.rand(4, 32, 14, 14)
print("a.shape", a.shape)

b = a.repeat(4, 32, 1, 1)
print("a.shape", a.shape)
print("b.shape", b.shape)

c = a.repeat(3, 2, 2, 2)
print("a.shape", a.shape)
print("c,shape", c.shape)

a.shape torch.Size([4, 32, 14, 14])
a.shape torch.Size([4, 32, 14, 14])
b.shape torch.Size([16, 1024, 14, 14])
a.shape torch.Size([4, 32, 14, 14])
c,shape torch.Size([12, 64, 28, 28])


In [9]:
# 变换4：二维张量转置操作 使用.t()方法，只适用于二维张量。 不改变原张量
a = torch.rand(4, 3)
print("a.shape", a.shape)

b = a.t()
print("b.shape", b.shape)
print("a.shape", a.shape)

a.shape torch.Size([4, 3])
b.shape torch.Size([3, 4])
a.shape torch.Size([4, 3])


In [16]:
# 变换5：使用transpose() API 交换维度
a = torch.rand(4, 3, 32, 32)
print("a.shape", a.shape)

b = a.transpose(1, 3)
print("b.shape", b.shape)

# .transpose()将数据的存储方式打乱了，数据不连续了，因此转置后的数据不能直接view变换
# c = b.view(4, 3 * 32 * 32)
# print("c,shape", c.shape)
 # 解决方案：使用 .contiguous()方法将数据变为连续存储
c = b.contiguous().view(4, 3 * 32 * 32)
print("c,shape", c.shape)

# transpose变换后的数据复原
a1 = a.transpose(1, 3).contiguous().view(4, 3 * 32 *32).view(4, 3, 32, 32)
a2 = a.transpose(1, 3).contiguous().view(4, 3 * 32 *32).view(4, 32, 32, 3).transpose(4, 3, 32, 32)

print(torch.all(torch.eq(a, a1)))
print(torch.all(torch.eq(a, a2))) 

a.shape torch.Size([4, 3, 32, 32])
b.shape torch.Size([4, 32, 32, 3])
c,shape torch.Size([4, 3072])


TypeError: transpose() received an invalid combination of arguments - got (int, int, int, int), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


In [ ]:
# 变换5：.permute() API